In [7]:
import models
import pandas as pd

In [8]:
MODEL = 'CNN_32_filters_5_kernels'
COMMODITY = 'cobalt_20'
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']

In [9]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
df = models.npy_to_top_n_f1_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'{COMMODITY}/test/predictions/test_F1_{algo[1]}', 10)
df

Confident 0.1Rule CNNA_128_3for CNN_32_5_predictions.csv
Confident 0.1Rule CNNA_32_3for CNN_32_5_predictions.csv
Confident 0.1Rule CNNA_32_5for CNN_32_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_128_3_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_128_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_128_7_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_256_3_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_256_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_256_7_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_32_3_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_32_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_32_7_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_64_3_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_64_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNNA_64_7_predictions.csv
Confident 0.1Rule CNN_32_5for CNN_32_5_predictions.csv
Confident 0.1Rule CNN_32_5for CNN_32_7_predictions.csv
Confident 0.1Rule CNN_32_5for LSTM_128_pred

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Confident 0.1Rule LSTM_256for CNN_32_5_predictions.csv
Confident 0.1Rule RNN_32for CNN_32_5_predictions.csv
Confident 0.2Rule CNNA_128_3for CNN_32_5_predictions.csv
Confident 0.2Rule CNNA_32_3for CNN_32_5_predictions.csv
Confident 0.2Rule CNNA_32_5for CNN_32_5_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_128_3_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_128_5_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_128_7_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_256_3_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_256_5_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_256_7_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_32_3_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_32_5_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_32_7_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_64_3_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_64_5_predictions.csv
Confident 0.2Rule CNN_32_5for CNNA_64_7_predictions.csv
Confident 0.2Rule CNN_32_5for CNN_32_5_predic

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Confident 0.95Rule CNN_32_5for CNNA_64_3_predictions.csv
Confident 0.95Rule CNN_32_5for CNNA_64_5_predictions.csv
Confident 0.95Rule CNN_32_5for CNNA_64_7_predictions.csv
Confident 0.95Rule CNN_32_5for CNN_32_5_predictions.csv
Confident 0.95Rule CNN_32_5for CNN_32_7_predictions.csv
Confident 0.95Rule CNN_32_5for LSTM_128_predictions.csv
Confident 0.95Rule CNN_32_5for LSTM_256_predictions.csv
Confident 0.95Rule CNN_32_5for LSTM_32_predictions.csv
Confident 0.95Rule CNN_32_5for LSTM_64_predictions.csv
Confident 0.95Rule CNN_32_5for RNN_128_predictions.csv
Confident 0.95Rule CNN_32_5for RNN_256_predictions.csv
Confident 0.95Rule CNN_32_5for RNN_32_predictions.csv
Confident 0.95Rule CNN_32_5for RNN_64_predictions.csv
Confident 0.95Rule CNN_32_7for CNN_32_5_predictions.csv
Confident 0.95Rule CNN_64_7for CNN_32_5_predictions.csv
Confident 0.95Rule LSTM_256for CNN_32_5_predictions.csv
Confident 0.95Rule RNN_32for CNN_32_5_predictions.csv
Confident 0.9Rule CNNA_128_3for CNN_32_5_predictions.cs

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Confident 0.9Rule CNN_32_5for CNNA_32_3_predictions.csv
Confident 0.9Rule CNN_32_5for CNNA_32_5_predictions.csv
Confident 0.9Rule CNN_32_5for CNNA_32_7_predictions.csv
Confident 0.9Rule CNN_32_5for CNNA_64_3_predictions.csv
Confident 0.9Rule CNN_32_5for CNNA_64_5_predictions.csv
Confident 0.9Rule CNN_32_5for CNNA_64_7_predictions.csv
Confident 0.9Rule CNN_32_5for CNN_32_5_predictions.csv
Confident 0.9Rule CNN_32_5for CNN_32_7_predictions.csv
Confident 0.9Rule CNN_32_5for LSTM_128_predictions.csv
Confident 0.9Rule CNN_32_5for LSTM_256_predictions.csv
Confident 0.9Rule CNN_32_5for LSTM_32_predictions.csv
Confident 0.9Rule CNN_32_5for LSTM_64_predictions.csv
Confident 0.9Rule CNN_32_5for RNN_128_predictions.csv
Confident 0.9Rule CNN_32_5for RNN_256_predictions.csv
Confident 0.9Rule CNN_32_5for RNN_32_predictions.csv
Confident 0.9Rule CNN_32_5for RNN_64_predictions.csv
Confident 0.9Rule CNN_32_7for CNN_32_5_predictions.csv
Confident 0.9Rule CNN_64_7for CNN_32_5_predictions.csv
Confident 0.

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
381,0,0,0,0,0,0,0,0,0
382,0,0,0,0,0,0,0,0,0
383,0,0,0,0,0,0,0,0,0
384,0,0,0,0,0,0,0,0,0


In [10]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [11]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.3835616438356165
recall: 0.2641509433962264
precision: 0.7


In [12]:
df.to_numpy().dump('edcr/data/test.npy')